In [ ]:
import sys
sys.version

In [ ]:
import numpy as np
import skimage.measure

In [ ]:
import pydicom
pydicom.__version__

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rai
rai.__version__

In [ ]:
import raicontours

from raicontours import TG263

raicontours.__version__

In [ ]:
cfg = raicontours.get_config()

In [ ]:
rai_starting_model, rai_dependent_model = rai.load_model(cfg=cfg)

In [ ]:
image_paths, structure_path = rai.download_deepmind_example()
structure_path

In [ ]:
x_grid, y_grid, image_stack, image_uids = rai.paths_to_image_stack_hfs(
    cfg=cfg, paths=image_paths
)

In [ ]:
image_stack.shape

In [ ]:
reduced_image_stack = skimage.measure.block_reduce(image_stack, block_size=cfg["reduce_block_sizes"][0], func=np.mean)
reduced_image_stack.shape

In [ ]:
310 / 4

In [ ]:
z = [15, 30]
y = [35, 60]
x = [50, 80]

predicted_masks = rai.inference_over_jittered_grid(
    cfg=cfg, model=rai_starting_model, grid=(z, y, x), image_stack=reduced_image_stack,
)

In [ ]:
# predicted_masks = rai.inference_over_jittered_grid(
#     cfg=cfg, model=rai_dependent_model, grid=(z, y, x), image_stack=reduced_image_stack, masks_stack=predicted_masks
# )

In [ ]:
cfg["reduce_block_sizes"]

In [ ]:
upscaled = predicted_masks

for i in range(3):
    upscaled = np.repeat(upscaled, repeats=2, axis=i)

if upscaled.shape[0] != image_stack.shape[0]:
    upscaled = upscaled.astype(np.float32)
    
    upscaled = np.mean([upscaled[1:, ...], upscaled[0:-1, ...]], axis=0)
    assert upscaled.shape[0] == image_stack.shape[0]
    
    upscaled = np.round(upscaled).astype(np.uint8)
    
upscaled.shape

In [ ]:
where_mask = np.where(upscaled > 127.5)
np.min(where_mask, axis=1)

In [ ]:
np.max(where_mask, axis=1)

In [ ]:
reduced_image_stack = skimage.measure.block_reduce(image_stack, block_size=cfg["reduce_block_sizes"][1], func=np.mean)
reduced_image_stack.shape

In [ ]:
z = [35, 45, 55]
y = [70, 95, 120]
x = [100, 130, 160]

predicted_masks = rai.inference_over_jittered_grid(
    cfg=cfg, model=rai_dependent_model, grid=(z, y, x), image_stack=reduced_image_stack, masks_stack=upscaled
)

In [ ]:
predicted_masks.shape

In [ ]:
np.max(predicted_masks)

In [ ]:
upscaled = predicted_masks

for i in range(1,3):
    upscaled = np.repeat(upscaled, repeats=2, axis=i)

if upscaled.shape[0] != image_stack.shape[0]:
    upscaled = upscaled.astype(np.float32)
    
    upscaled = np.mean([upscaled[1:, ...], upscaled[0:-1, ...]], axis=0)
    assert upscaled.shape[0] == image_stack.shape[0]
    
    upscaled = np.round(upscaled).astype(np.uint8)
    
upscaled.shape

In [ ]:
z = [45, 50, 55]
y = [140, 155, 175, 195, 215, 230]
x = [210, 230, 250, 270, 290, 310]

predicted_masks = rai.inference_over_jittered_grid(
    cfg=cfg, model=rai_dependent_model, grid=(z, y, x), image_stack=image_stack, masks_stack=upscaled
)

In [ ]:
predicted_contours_by_structure = rai.masks_to_contours_by_structure(
    cfg=cfg, x_grid=x_grid, y_grid=y_grid, masks=predicted_masks
)

predicted_contours_by_structure.keys()

In [ ]:
rai.plot_contours_by_structure(
    x_grid, y_grid, image_stack, predicted_contours_by_structure
)

## Compare to DICOM-RT Structure

In [ ]:
structure_ds = pydicom.read_file(structure_path)
[item.ROIName for item in structure_ds.StructureSetROISequence]

In [ ]:
align_map = {
    "Orbit-Lt": [TG263.Eye_L],
    "Orbit-Rt": [TG263.Eye_R],
    "Lacrimal-Lt": [TG263.Glnd_Lacrimal_L],
    "Lacrimal-Rt": [TG263.Glnd_Lacrimal_R],
    "Lens-Lt": [TG263.Lens_L],
    "Lens-Rt": [TG263.Lens_R],
    "Optic-Nerve-Lt": [TG263.OpticNrv_L],
    "Optic-Nerve-Rt": [TG263.OpticNrv_R],
}

In [ ]:
structure_names = list(align_map.keys())
structure_names

In [ ]:
dicom_contours_by_structure = rai.dicom_to_contours_by_structure(
    ds=structure_ds, image_uids=image_uids, structure_names=structure_names
)
dicom_contours_by_structure.keys()

In [ ]:
aligned_predicted_contours_by_structure = rai.merge_contours_by_structure(
    predicted_contours_by_structure, align_map
)
aligned_predicted_contours_by_structure.keys()

In [ ]:
dice = {}
for name in align_map:
    dice[name] = rai.dice_from_contours_by_slice(
        dicom_contours_by_structure[name],
        aligned_predicted_contours_by_structure[name],
    )

dice

In [ ]:
# {'Orbit-Lt': 0.9362643339720585,
#  'Orbit-Rt': 0.9118221819034799,
#  'Lacrimal-Lt': 0.5370672348555429,
#  'Lacrimal-Rt': 0.7410031278293367,
#  'Lens-Lt': 0.6199596069529278,
#  'Lens-Rt': 0.5651545326203434,
#  'Optic-Nerve-Lt': 0.6357191302295039,
#  'Optic-Nerve-Rt': 0.682449321749098}

In [ ]:
combined_contours_by_structure = {
    **predicted_contours_by_structure,
    **dicom_contours_by_structure,
}

rai.plot_contours_by_structure(
    x_grid, y_grid, image_stack, combined_contours_by_structure, align_map
)